In [1]:
import os
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
import myServices as ms
import models as md
import numpy as np
import pandas as pd
import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,roc_curve, auc, roc_auc_score, f1_score
import joblib

In [2]:
# to compute ececution time do: 
# with timeit():
#     # your code, e.g., 
class timeit(): 
    from datetime import datetime
    def __enter__(self):
        self.tic = self.datetime.now()
    def __exit__(self, *args, **kwargs):
        print('runtime: {}'.format(self.datetime.now() - self.tic))

## Importing and manipulating datasets

In [ ]:
sklearn.metrics.get_scorer_names()

In [3]:
### Cleaning basin1DataSet 
dataSetPath = 'datasets/basin4_RawData.csv'
basinDataSet = pd.read_csv(dataSetPath, index_col = None)
# basin1Light = pd.read_csv('datasetBasin1_NoDataFree.csv', index_col = None)
print(basinDataSet.info())
basinDataSet.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279641 entries, 0 to 279640
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   percentage  15956 non-null   float64
 1   visibility  279641 non-null  float64
 2   TPI         277252 non-null  float64
 3   TWI         279641 non-null  float64
 4   LDSOL5R200  269307 non-null  float64
 5   LDSOL5R150  269307 non-null  float64
 6   LDSOL4R150  264969 non-null  float64
 7   FAProx025   279641 non-null  float64
 8   FAProx01    279641 non-null  float64
 9   slope       279636 non-null  float64
 10  elevation   279641 non-null  float64
 11  x_coord     279641 non-null  int64  
 12  y_coord     279641 non-null  int64  
dtypes: float64(11), int64(2)
memory usage: 27.7 MB
None


,percentage,visibility,TPI,TWI,LDSOL5R200,LDSOL5R150,LDSOL4R150,FAProx025,FAProx01,slope,elevation,x_coord,y_coord
count,15956.000000,279641.000000,277252.000000,279641.000000,269307.000000,269307.000000,264969.000000,279641.000000,279641.000000,279636.000000,279641.000000,279641.000000,2.796410e+05
mean,4.168714,0.061356,-0.000226,9.311844,0.006653,0.006699,0.003132,272.570003,181.913610,6.353948,126.258612,361059.731023,5.273251e+06
std,1.623044,0.048661,0.202981,4.026503,0.003409,0.004169,0.003029,201.496830,131.952031,5.963830,30.371440,922.890858,1.112358e+03
min,1.000000,0.000050,-4.801120,-6.874480,0.000000,0.000000,0.000000,0.000000,0.000000,0.000390,-10.448000,359138.000000,5.270818e+06
25%,5.000000,0.027310,-0.078290,5.699740,0.004060,0.003400,0.000000,110.453610,74.330350,2.285347,118.822280,360253.000000,5.272333e+06
50%,5.000000,0.047120,-0.000820,10.311960,0.006530,0.006400,0.002570,232.594070,156.604600,4.910465,132.357010,361113.000000,5.273358e+06
75%,5.000000,0.082310,0.078480,12.538330,0.009020,0.009550,0.005360,395.000000,268.700590,8.516732,142.956530,361838.000000,5.274063e+06
max,5.000000,0.448310,4.880940,22.801570,0.019030,0.021440,0.012770,975.140990,675.740360,67.607030,205.594620,362883.000000,5.275618e+06


In [4]:
basinDataSet.isna().any()

percentage     True
visibility    False
TPI            True
TWI           False
LDSOL5R200     True
LDSOL5R150     True
LDSOL4R150     True
FAProx025     False
FAProx01      False
slope          True
elevation     False
x_coord       False
y_coord       False
dtype: bool

In [59]:
basinDataSet['LDSOL4R200'].fillna(0,inplace=True)

In [ ]:
basinDataSet.dropna(subset=['TPI'],inplace=True)

In [61]:
basinDataSet.isna().sum()

percentage    109798
TWI                0
TPI                0
LDSOL3R150         0
LDSOL4R150         0
LDSOL4R200         0
FAProx_01          0
FAProx_025         0
visibility         0
slope              0
elevation          0
x_coord            0
y_coord            0
dtype: int64

In [ ]:
basinDataSet.dropna(inplace=True)

In [66]:
basinDataSet['percentage'].fillna(0,inplace=True)  # ,'DLSOL5R150','DLSOL5R200'

In [67]:
basinDataSet.isna().sum()
# print(basinDataSet['FAcc'].max())

percentage    0
TWI           0
TPI           0
LDSOL3R150    0
LDSOL4R150    0
LDSOL4R200    0
FAProx_01     0
FAProx_025    0
visibility    0
slope         0
elevation     0
x_coord       0
y_coord       0
dtype: int64

In [ ]:
#### NOrmalize Flow Accumulation
basinDataSet['FAcc'] = (basinDataSet['FAcc']- basinDataSet['FAcc'].min())/(basinDataSet['FAcc'].max()-basinDataSet['FAcc'].min())


In [ ]:
### Replacing with 0 
repalcer  = basinDataSet['FAProx_01'].to_numpy()
basinDataSet['FAProx_01'] = [0 if repalcer[j] == -9999 else repalcer[j] for j in range(len(repalcer))]                                                                                                                         
                                                                                                                          

In [72]:
## Transform datatype of a column
repalcer  = basinDataSet['percentage'].to_numpy().astype('int')
basinDataSet.loc[:,'percentage'] = repalcer

In [63]:
basinDataSet.describe()

,percentage,TWI,TPI,LDSOL3R150,LDSOL4R150,LDSOL4R200,FAProx_01,FAProx_025,visibility,slope,elevation,x_coord,y_coord
count,2984.000000,112782.000000,112782.000000,112782.000000,112782.000000,112782.000000,112782.000000,112782.000000,112782.000000,112782.000000,112782.000000,112782.000000,1.127820e+05
mean,4.182306,9.426335,-0.000350,0.003837,0.007590,0.007421,92.264157,27.500874,0.114898,6.303644,123.901570,359743.982426,5.273088e+06
std,1.613390,4.067111,0.208581,0.003375,0.004548,0.003981,74.957074,21.349713,0.076658,5.480844,27.467651,456.708242,6.424637e+02
min,1.000000,-5.067620,-2.566120,0.000000,0.000000,0.000000,0.000000,0.000000,0.000160,0.005530,0.372080,358698.000000,5.271343e+06
25%,5.000000,5.777007,-0.087020,0.000400,0.004220,0.004340,31.622780,10.198040,0.053740,2.722058,106.734668,359383.000000,5.272668e+06
50%,5.000000,10.394990,-0.001010,0.003750,0.007550,0.007480,74.330350,22.803510,0.104230,4.991620,122.139580,359763.000000,5.273128e+06
75%,5.000000,12.656813,0.085148,0.005570,0.010970,0.010290,136.014710,40.521600,0.164190,8.282625,145.815840,360083.000000,5.273553e+06
max,5.000000,21.760160,2.504120,0.016930,0.023630,0.018680,409.175990,114.280360,0.499120,50.978400,200.815220,360758.000000,5.274413e+06


In [73]:
basinDataSet.to_csv('datasets/basin3_CleanDataSet.csv', index=None)

In [74]:
### Making quadratic transformation in Labels for regression
DS = pd.read_csv('datasets/basin3_CleanDataSet.csv', index_col = None)
print(Counter(DS['percentage']))

# y_Quad = md.quadraticRechapeLabes(DS['percentage'], -0.125, 0.825)
# DS['percentage'] = y_Quad
# print(Counter(DS['percentage']))
# DS.to_csv('basin1TrainUnbalanced_QuadTarget.csv', index = None)

Counter({0: 109798, 5: 2374, 1: 610})


In [ ]:
ds = DS.head(5)
s = {}
s['Datas'] = ds
print(s)

## balanced sampling

In [75]:
## Stratified Split
from sklearn.model_selection import StratifiedShuffleSplit

X,Y = ms.importDataSet('datasets/basin3_CleanDataSet.csv', 'percentage')
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=50)
for train_index, test_index in sss.split(X, Y):
    print("TRAIN:", train_index.size, "TEST:", test_index.size)
    X_train = X.iloc[train_index]
    y_train = Y.iloc[train_index]
    X_test = X.iloc[test_index]
    y_test = Y.iloc[test_index]

TRAIN: 90225 TEST: 22557


In [76]:
print(len(X_train['elevation']), len(y_train) )
Counter(y_train)

90225 90225


Counter({0: 87838, 5: 1899, 1: 488})

In [77]:
#####. Creating training set
X_train.loc[:,'percentage'] = y_train
print(X_train.head())

             TWI      TPI  LDSOL3R150  LDSOL4R150  LDSOL4R200  FAProx_01  \
106195  13.59353 -0.15649     0.00315     0.00693     0.00671   85.00000   
32043    4.63718 -0.00439     0.00415     0.01119     0.01205   55.00000   
89371   11.71443 -0.41978     0.00092     0.00764     0.01099  125.00000   
112187  11.53867  0.02208     0.00000     0.00390     0.00264  114.01755   
23716    4.67272  0.19862     0.00379     0.00568     0.00567   93.00538   

        FAProx_025  visibility     slope  elevation  x_coord  y_coord  \
106195    25.94224     0.04296   9.59954  159.47165   359848  5271963   
32043     11.18034     0.09969   6.23723   94.90872   359373  5273488   
89371     29.96665     0.07519  10.90686  134.19553   360123  5272568   
112187    72.53275     0.02210   4.82951  178.78873   359813  5271498   
23716     19.79899     0.20469   8.11566   86.85973   358933  5273638   

        percentage  
106195           0  
32043            0  
89371            0  
112187           0  

/var/folders/s5/cqkwnnps0f1_35_bccj0y8q80000gn/T/ipykernel_9529/1088689243.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.loc[:,'percentage'] = y_train


In [78]:
print(X_train.head())
Counter(X_train['percentage'])

             TWI      TPI  LDSOL3R150  LDSOL4R150  LDSOL4R200  FAProx_01  \
106195  13.59353 -0.15649     0.00315     0.00693     0.00671   85.00000   
32043    4.63718 -0.00439     0.00415     0.01119     0.01205   55.00000   
89371   11.71443 -0.41978     0.00092     0.00764     0.01099  125.00000   
112187  11.53867  0.02208     0.00000     0.00390     0.00264  114.01755   
23716    4.67272  0.19862     0.00379     0.00568     0.00567   93.00538   

        FAProx_025  visibility     slope  elevation  x_coord  y_coord  \
106195    25.94224     0.04296   9.59954  159.47165   359848  5271963   
32043     11.18034     0.09969   6.23723   94.90872   359373  5273488   
89371     29.96665     0.07519  10.90686  134.19553   360123  5272568   
112187    72.53275     0.02210   4.82951  178.78873   359813  5271498   
23716     19.79899     0.20469   8.11566   86.85973   358933  5273638   

        percentage  
106195           0  
32043            0  
89371            0  
112187           0  

Counter({0: 87838, 5: 1899, 1: 488})

In [79]:
X_train.drop(['x_coord','y_coord'], axis =1, inplace=True)
X_train.head()

/var/folders/s5/cqkwnnps0f1_35_bccj0y8q80000gn/T/ipykernel_9529/2609724293.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.drop(['x_coord','y_coord'], axis =1, inplace=True)


,TWI,TPI,LDSOL3R150,LDSOL4R150,LDSOL4R200,FAProx_01,FAProx_025,visibility,slope,elevation,percentage
106195,13.59353,-0.15649,0.00315,0.00693,0.00671,85.00000,25.94224,0.04296,9.59954,159.47165,0
32043,4.63718,-0.00439,0.00415,0.01119,0.01205,55.00000,11.18034,0.09969,6.23723,94.90872,0
89371,11.71443,-0.41978,0.00092,0.00764,0.01099,125.00000,29.96665,0.07519,10.90686,134.19553,0
112187,11.53867,0.02208,0.00000,0.00390,0.00264,114.01755,72.53275,0.02210,4.82951,178.78873,0
23716,4.67272,0.19862,0.00379,0.00568,0.00567,93.00538,19.79899,0.20469,8.11566,86.85973,0


In [80]:
X_train.to_csv('datasets/basin3_Training.csv', index=None)

In [81]:
#####. Creating training set
print(X_test.head())
X_test.loc[:,'percentage'] = y_test
print(X_test.head())
print(X_test.info())


            TWI      TPI  LDSOL3R150  LDSOL4R150  LDSOL4R200  FAProx_01  \
41078  11.78652 -0.02912     0.00497     0.01353     0.01384  101.24229   
76818  11.09787 -0.06277     0.01116     0.01408     0.01234   35.35534   
86254   2.85848  0.07752     0.00791     0.01357     0.01096   60.00000   
28142   1.70899  0.24947     0.00482     0.01076     0.01006   49.49747   
19505  15.95014 -0.02366     0.00517     0.00996     0.00884   30.41381   

       FAProx_025  visibility    slope  elevation  x_coord  y_coord  
41078    26.47640     0.18092  7.61512  121.83397   360003  5273348  
76818     7.81025     0.10706  2.43555  116.16261   359898  5272813  
86254    12.00000     0.10051  0.68381  123.42653   359863  5272633  
28142     9.89950     0.11470  4.03959   96.31892   359478  5273553  
19505    32.24903     0.06111  3.11087  103.15604   359578  5273738  
            TWI      TPI  LDSOL3R150  LDSOL4R150  LDSOL4R200  FAProx_01  \
41078  11.78652 -0.02912     0.00497     0.01353     0

/var/folders/s5/cqkwnnps0f1_35_bccj0y8q80000gn/T/ipykernel_9529/2312397218.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.loc[:,'percentage'] = y_test


In [82]:
Counter(X_test['percentage'])

Counter({0: 21960, 5: 475, 1: 122})

In [87]:
X_test.to_csv('datasets/basin3_Test.csv', index=None)

In [84]:
## This proportions are the reason why a sample_weight of 0.01 for the majority class give best results for regression
totalTrain = sum([87838, 1899, 488])
totalValidation = sum([21960,475, 122])
print("Summary of traning and test dataset class balance")
print(f"Training Set:", '\n', "Class 0: %.3f" %(87838/totalTrain), " Class 1: %.4f" %(488/totalTrain), "Class 5: %.4f"%(1899/totalTrain))
print("Testing Set:", '\n', "Class 0: %.3f" %(21960/totalValidation)," Class 1: %.4f" %(122/totalValidation),  "Class 5: %.4f"%(475/totalValidation))



Summary of traning and test dataset class balance
Training Set: 
 Class 0: 0.974  Class 1: 0.0054 Class 5: 0.0210
Testing Set: 
 Class 0: 0.974  Class 1: 0.0054 Class 5: 0.0211


In [86]:
print(totalTrain)

90225


In [ ]:
model = ms.loadModel('./outputs/2022-08-05/00-35-58/2208050035.pkl')
dataSetToSave = ms.makePredictionToImportAsSHP(csvName, model, X, Y, 'percentage')
print(dataSetToSave.head())

## Controled sampling

In [ ]:
DS = pd.read_csv('basin1Light_Clean_Training.csv', index_col = None)
print(DS.head())


In [ ]:
print(DS.columns)

In [ ]:
plt.boxplot(DS['FAcc']) # , , DS['elevation'], DS['disToRiv']]

In [ ]:
## Resampling appliying class selection by rule:

# RULE1: Select point at a distance to river less than 300m. 

# # newDS = pseudoClassCreation(DS, "distanceToRiver", 300, 2)
def pseudoClassCreation(dataset, conditionVariable, threshold, pseudoClass, targetClassName):
    '''
    Replace <targetClass> by  <pseudoClass> where <conditionVariable >= threshold>. 
    Return:
      dataset with new classes group. 
    '''
    datsetReclassified = dataset.copy()
    actualTarget = (np.array(dataset[targetClassName])).ravel()
    conditionVar = (np.array(dataset[conditionVariable])).ravel()
    datsetReclassified[targetClassName] = [ pseudoClass if conditionVar[j] >= threshold 
                                           else actualTarget[j]
                                           for j in range(len(actualTarget))]
    print(Counter(datsetReclassified[targetClassName]))
    return  datsetReclassified

def revertPseudoClassCreation(dataset, originalClass, pseudoClass, targetClassName):
    '''
    Restablich  <targetClass> with <originalClass> where <targetClassName == pseudoClass>. 
    Return:
      dataset with original classes group. 
    '''
    datsetReclassified = dataset.copy()
    actualTarget = (np.array(dataset[targetClassName])).ravel()
    datsetReclassified[targetClassName] = [ originalClass if actualTarget[j] == pseudoClass
                                           else actualTarget[j]
                                           for j in range(len(actualTarget))]
    print(Counter(datsetReclassified[targetClassName]))
    return  datsetReclassified


print(Counter(X_train['percentage']))
newDS = pseudoClassCreation(X_train, 'disToRiv', 200, 2, 'percentage')
y = newDS['percentage']
newDS.drop(['percentage'], axis=1, inplace = True)
x_res,y_res = ms.randomUndersampling(newDS, y, )
x_res['percentage'] = y_res
# newDatase = revertPseudoClassCreation(x_res, 0, 2, 'percentage')


In [ ]:
x_res.to_csv('basin1ControlClass0Sampling4Class_ToSHP.csv',index = None)

# Data description and visualization

In [ ]:
#### import dataset to describe

DS= pd.read_csv('datasets/basin1Light_Clean.csv', index_col=None)
DS.head()

In [ ]:
DS.drop(['x_coord','y_coord'], axis = 1, inplace=True)
DS.head()

In [ ]:
### FAcc vs Labels
targets = DS['percentage']
FAcc = original['FAcc']
FAcc_norm = DS['FAcc_norm']
fig, axs = plt.subplots(1, 2, figsize=(13,4), sharey=True)
fig.text(-0.02, 0.5, 'labels', va='center', rotation='vertical')
fig.text(0.5, 1, 'Flow accumulation vs labels distribution', ha ='center')
axs[0].scatter(FAcc,targets)
# axs[0].set_title("Facc")
axs[0].set(xlabel='a) Flow Accumulation')
axs[1].scatter(FAcc_norm,targets)
# axs[1].set_title("FAcc_norm")
axs[1].set(xlabel='b) Flow Accumulation estandardized')
plt.rcParams['font.size'] = '20'
fig.tight_layout()


In [ ]:

## Plot all features vs labels
# 'disToRiv', 'TWI', 'TPI', 'slope', 'elevation',

targets = DS['percentage']
# targets = np.where(targets == 5,2,targets)

E = DS['elevation'] 
slope = DS['slope']
FAcc = DS['FAcc']
TWI = DS['TWI']
TPI = DS['TPI']
DLSOL4R150 = DS['DLSOL4R150']
DLSOL5R150 = DS['DLSOL5R150']
DLSOL5R200 = DS['DLSOL5R200']
FAProx_01 = DS['FAProx_01']
FAProx_025 = DS['FAProx_025']
visibility = DS['visibility']

fig, axs = plt.subplots(4,3, figsize=(13, 8), sharey=True)
fig.supylabel('Labels')
plt.rcParams['font.size'] = '15'
plt.yticks([0,1,5])

'''
E = DS['elevation'] 
slope = DS['slope']
FAcc = DS['FAcc']
TWI = DS['TWI']
'''
axs[0, 0].scatter(E,targets)
axs[0, 0].set_title("Elevation")
axs[1, 0].scatter(slope,targets)
axs[1, 0].set_title("Slope")
axs[2, 0].scatter(FAcc,targets)
axs[2, 0].set_title("Flow accumulation")
axs[3, 0].scatter(TWI,targets)
axs[3, 0].set_title("TWI")

'''
TPI = DS['TPI']
DLSOL4R150 = DS['DLSOL4R150']
DLSOL5R150 = DS['DLSOL5R150']
DLSOL5R200 = DS['DLSOL5R200']
'''
axs[0, 1].scatter(TPI,targets)
axs[0, 1].set_title('TPI')
axs[1, 1].scatter(DLSOL4R150,targets)
axs[1, 1].set_title("DLSOL4R150")
axs[2, 1].scatter(DLSOL5R150,targets)
axs[2, 1].set_title("DLSOL5R150")
axs[3, 1].scatter(DLSOL5R200,targets)
axs[3, 1].set_title("DLSOL5R200")

'''
FAProx_01 = DS['FAProx_01']
FAProx_025 = DS['FAProx_025']
visibility = DS['visibility']
'''
axs[0, 2].scatter(FAProx_01,targets)
axs[0, 2].set_title('FAProx_01')
axs[1, 2].scatter(FAProx_025,targets)
axs[1, 2].set_title("FAProx_025")
axs[2, 2].scatter(visibility,targets)
axs[2, 2].set_title("Visibility")

fig.tight_layout()


In [ ]:
print(DS.head())
#  Return a dataset with the rows corresponding to the index where condition in DS.columName is valid. 
dsArray = DS[DS.percentage != 0] print(dsArray.head())
print(dsArray.head()) 

In [ ]:
sns.set(font_scale=1.5)
sns.pairplot(DS, hue = 'percentage', diag_kind = 'kde', 
             plot_kws = {'alpha': 0.8, 's': 100},
             height = 4, corner=True, palette = "Set2")# vars = ['life_exp', 'log_pop', 'log_gdp_per_cap'],

# sns.pairplot(DS, hue="percentage")

In [ ]:
####. Covariance Matrix
sns.set(font_scale=0.7)
matrix = DS.corr().round(2)
sns.heatmap(matrix, annot=True)
plt.set_figsize=(25,20)
plt.show()

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier

estimator = RandomForestClassifier(criterion='entropy', random_state = 50)
x_train,y_train = ms.importDataSet('basin1Train.csv', 'percentage')
classifier = OneVsRestClassifier(estimator).fit(x_train,y_train)


In [ ]:
classifier = ms.loadModel('./outputs/2022-08-05/11-01-57/2208051101.pkl')
x_test,y_test = ms.importDataSet('basin1Test.csv', 'percentage')

x_test = ms.removeCoordinatesFromDataSet(x_test)

# y_prob = classifier.predict_proba(x_test)
#print(np.unique(y_prob))

In [ ]:
md.plot_ROC_AUC_OneVsRest(classifier, x_test, y_test)

In [ ]:
_,y_test = ms.importDataSet('./bestModels/Classifier/10-18-08/2208051018prediction_basin1Test.csv', 'prediction')
unique, count = np.unique(y_test, return_counts=True)
total = count.sum()
print(total)
percent = np.round(np.zeros_like(unique).astype('float16'),3)
print('values, counts , percent')
for i in range(len(unique)):    
   percent[i] = (count[i]/total)*100
   print(unique[i],"\t", count[i], percent[i])
